In [1]:
import os
from glob import glob
import pandas as pd
import json

In [2]:
speaker = "system"
max_optional_slots = 2 # 前回生成した条件
task_batch_size = 20 # 1タスクにつき20個のDA
group_batch_size = 50 # 1グループにつき50タスク

da_list = pd.DataFrame(json.load(open(f"da_list-{speaker}-max{max_optional_slots}/all.json")))

In [3]:
# 第一回で収集されなかったDA
result_dpath = "result-max2"
result = pd.concat([pd.read_csv(result_fpath) for result_fpath in glob(os.path.join(result_dpath, f"{speaker}*.csv"))])
all_ids = da_list["id"].unique()
completed_ids = result["id"].unique()
incompleted_ids = set(all_ids) - set(completed_ids)
incompleted_da_list = da_list[da_list['id'].isin(incompleted_ids)]
incompleted_da_list = json.loads(incompleted_da_list.to_json(orient='records'))

In [4]:
# slotの数が0もしくは1つしかないDA
## generate_fewer_slots_da_list.ipynbで再生成する
fewer_da_list = json.load(open(f"fewer_slots_da_list-{speaker}-max2/all.json"))

In [5]:
da_list_2nd = incompleted_da_list + fewer_da_list

da_list_dpath = f"da_list-2nd-{speaker}-max{max_optional_slots}"
os.makedirs(da_list_dpath, exist_ok=True)
config = {
    "speaker": speaker,
    "max_optional_slots": max_optional_slots,
    "task_batch_size": task_batch_size,
    "group_batch_size": group_batch_size,
    "total_da": len(da_list_2nd)
}
json.dump(config, open(f"{da_list_dpath}/meta_info.json", "w"), indent=4)
json.dump(da_list_2nd, open(f"{da_list_dpath}/all.json", "w"), indent=4, ensure_ascii=False)

from util import split_da_list
groups = split_da_list(da_list=da_list_2nd, task_batch_size=task_batch_size, group_batch_size=group_batch_size)
for g_i, group in enumerate(groups):
    group_dpath = f"{da_list_dpath}/group{g_i}"
    os.makedirs(group_dpath, exist_ok=True)
    for t_i, task in enumerate(group):
        json.dump(task, open(f"{group_dpath}/task{t_i}.json", "w"), indent=4, ensure_ascii=False)